In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 350
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-only-stats-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\350 (134s),  symKL_DB_eta: 345.084,  symKL_DB_z: 389.030,  loss: -1471.152,  ess: 3.020,  kl_eta_ex: 993.368,  kl_eta_in: 160.669,  kl_z_ex: 275.248,  kl_z_in: 36.137
epoch: 1\350 (134s),  symKL_DB_eta: 194.762,  symKL_DB_z: 193.710,  loss: -1031.986,  ess: 3.375,  kl_eta_ex: 494.740,  kl_eta_in: 124.321,  kl_z_ex: 123.027,  kl_z_in: 25.824
epoch: 2\350 (129s),  symKL_DB_eta: 123.486,  symKL_DB_z: 114.452,  loss: -826.956,  ess: 3.681,  kl_eta_ex: 280.960,  kl_eta_in: 103.071,  kl_z_ex: 64.205,  kl_z_in: 19.236
epoch: 3\350 (131s),  symKL_DB_eta: 82.680,  symKL_DB_z: 81.932,  loss: -721.374,  ess: 3.978,  kl_eta_ex: 181.547,  kl_eta_in: 94.379,  kl_z_ex: 42.632,  kl_z_in: 14.919
epoch: 4\350 (134s),  symKL_DB_eta: 60.220,  symKL_DB_z: 63.654,  loss: -654.154,  ess: 4.283,  kl_eta_ex: 126.288,  kl_eta_in: 84.717,  kl_z_ex: 31.531,  kl_z_in: 11.577
epoch: 5\350 (135s),  symKL_DB_eta: 44.235,  symKL_DB_z: 51.018,  loss: -597.720,  ess: 4.548,  kl_eta_ex: 86.827,  kl_eta_in: 71.73

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)